# 0. Import Pacakges

In [24]:
!pip install mediapipe opencv-python pandas scikit-learn
#opencv: webccam 결과를 보여주기 위해 사용
#pandas: data tool
# scikit-learn: machine learning tool (to train)
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv #좌표값들을 numpy array 형태로 저장하기 위해 csv module import
import os
import numpy as np

In [25]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# 1. Make Some Detections

In [26]:
# webcam에 연결 (보통 웹캠은 index 0으로 연결이 되어있으며, 
#error가 뜨는 경우 다른 숫자를 입력해 찾는다, 저장된 동영상을 넣고 싶은 경오 경로를 넣어주면 됨)
cap = cv2.VideoCapture(0) 
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # while 문을 사용->webcam이 실행중이면 아래 반복문 실행하여 holicstic model을 이용
    while cap.isOpened():
        ret, frame = cap.read()
        
        # webcam을 처음 읽으면 bgr로 인식이 되므로 mediapipe를 사용하려면 rgb순으로 바꿔줘야 한다.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #wireable flag를 우선 false로 해놓고 detection을 한 후 True로 변경       
        
        # Make Detections
        results = holistic.process(image)#image를 holisctic 모델에 넣어 탐지한 결과를 result 변수에 저장
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True  #detection 후에 True로 바꿔준다. 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks # landmarks 458개
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
# webcam을 닫는다.
cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<img src="https://i.imgur.com/AzKNp7A.png">-->

In [27]:
print(len(results.pose_landmarks.landmark))
print(len(results.face_landmarks.landmark))

num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark) # pose landmark 33, facelandmark 468개
num_coords

33
468


501

In [5]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [42]:
with open('tesla.csv', mode='w', newline='') as f: # csv 확장자파일 coords를 쓰기 형태로 연다.
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [45]:
class_name = "on handling"

In [46]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # pose landmark를 추출 후 numpy array 형태로 변환해준다. 이 때 flatten을 사용해 1차원으로 만들어준다.
            pose = results.pose_landmarks.landmark 
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # pose 열과 face 열의 정보들을 이어준다.
            row = pose_row+face_row
            
            #  위에서 정의한 class name을 입력해준다.
            row.insert(0, class_name)
            
            # Export to CSV
            with open('tesla.csv', mode='a', newline='') as f:# 이미 만들어진 csv파일에 새로운 데이터를 입력하는 형태로 열어준다.
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [6]:
import pandas as pd 
from sklearn.model_selection import train_test_split #train dataset과 tset dataset으로 나누는 함수 import

In [7]:
df = pd.read_csv('tesla.csv') # coor.csv로 부터 data를 읽어온다.

In [8]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,no handling,0.580053,0.501478,-0.694266,1.0,0.598859,0.468517,-0.604791,1.0,0.613672,...,0.016364,0.0,0.616431,0.475470,0.034452,0.0,0.620202,0.471000,0.035626,0.0
1,no handling,0.577115,0.503162,-0.888272,1.0,0.599318,0.470178,-0.795386,1.0,0.614810,...,0.015671,0.0,0.620444,0.473644,0.034043,0.0,0.624133,0.470088,0.035086,0.0
2,no handling,0.578185,0.504027,-0.820763,1.0,0.600005,0.471039,-0.732808,1.0,0.615097,...,0.016021,0.0,0.622103,0.479281,0.034743,0.0,0.625970,0.475309,0.035934,0.0
3,no handling,0.580567,0.507871,-0.741323,1.0,0.601659,0.473851,-0.655580,1.0,0.616745,...,0.014927,0.0,0.620831,0.484859,0.031953,0.0,0.624880,0.480526,0.032944,0.0
4,no handling,0.580704,0.510263,-0.829618,1.0,0.601796,0.476005,-0.740312,1.0,0.616884,...,0.015714,0.0,0.620579,0.487793,0.034107,0.0,0.624449,0.483998,0.035229,0.0


In [9]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
666,on handling,0.571369,0.540397,-0.505271,1.0,0.589500,0.509038,-0.431931,1.0,0.602413,...,0.012607,0.0,0.604125,0.522324,0.030504,0.0,0.607733,0.518513,0.031673,0.0
667,on handling,0.570394,0.545358,-0.515184,1.0,0.589085,0.511934,-0.444742,1.0,0.603023,...,0.013173,0.0,0.605028,0.522678,0.031351,0.0,0.608762,0.518855,0.032517,0.0
668,on handling,0.570073,0.548237,-0.737103,1.0,0.589924,0.514385,-0.668750,1.0,0.603804,...,0.013290,0.0,0.605621,0.519495,0.031548,0.0,0.609251,0.516023,0.032745,0.0
669,on handling,0.571479,0.547610,-0.777668,1.0,0.592841,0.516103,-0.706545,1.0,0.606872,...,0.015099,0.0,0.607908,0.512915,0.031962,0.0,0.611908,0.508589,0.033209,0.0
670,on handling,0.571463,0.537599,-0.729033,1.0,0.593037,0.509992,-0.651079,1.0,0.607025,...,0.014674,0.0,0.607841,0.499281,0.031139,0.0,0.611756,0.495435,0.032281,0.0


In [51]:
df[df['class']=='on handling']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
227,on handling,0.565637,0.629227,-0.213324,1.0,0.583925,0.599419,-0.137044,1.0,0.597350,...,0.013250,0.0,0.606526,0.601335,0.030816,0.0,0.609932,0.597492,0.031844,0.0
228,on handling,0.566504,0.627656,-0.144852,1.0,0.587023,0.601657,-0.067273,1.0,0.600617,...,0.009759,0.0,0.604004,0.597280,0.025802,0.0,0.607430,0.593815,0.026561,0.0
229,on handling,0.567522,0.630680,-0.262460,1.0,0.588050,0.603664,-0.191734,1.0,0.601579,...,0.011733,0.0,0.601863,0.600274,0.028410,0.0,0.605458,0.596664,0.029266,0.0
230,on handling,0.563519,0.632943,-0.323639,1.0,0.585034,0.602600,-0.253127,1.0,0.599123,...,0.010880,0.0,0.601025,0.598665,0.027878,0.0,0.604343,0.595473,0.028748,0.0
231,on handling,0.560457,0.633722,-0.337811,1.0,0.582649,0.602199,-0.270480,1.0,0.596985,...,0.010832,0.0,0.603145,0.601311,0.028102,0.0,0.606640,0.598158,0.029016,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,on handling,0.571369,0.540397,-0.505271,1.0,0.589500,0.509038,-0.431931,1.0,0.602413,...,0.012607,0.0,0.604125,0.522324,0.030504,0.0,0.607733,0.518513,0.031673,0.0
667,on handling,0.570394,0.545358,-0.515184,1.0,0.589085,0.511934,-0.444742,1.0,0.603023,...,0.013173,0.0,0.605028,0.522678,0.031351,0.0,0.608762,0.518855,0.032517,0.0
668,on handling,0.570073,0.548237,-0.737103,1.0,0.589924,0.514385,-0.668750,1.0,0.603804,...,0.013290,0.0,0.605621,0.519495,0.031548,0.0,0.609251,0.516023,0.032745,0.0
669,on handling,0.571479,0.547610,-0.777668,1.0,0.592841,0.516103,-0.706545,1.0,0.606872,...,0.015099,0.0,0.607908,0.512915,0.031962,0.0,0.611908,0.508589,0.033209,0.0


In [10]:
df[df['class']=='no handling']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,no handling,0.580053,0.501478,-0.694266,1.0,0.598859,0.468517,-0.604791,1.0,0.613672,...,0.016364,0.0,0.616431,0.475470,0.034452,0.0,0.620202,0.471000,0.035626,0.0
1,no handling,0.577115,0.503162,-0.888272,1.0,0.599318,0.470178,-0.795386,1.0,0.614810,...,0.015671,0.0,0.620444,0.473644,0.034043,0.0,0.624133,0.470088,0.035086,0.0
2,no handling,0.578185,0.504027,-0.820763,1.0,0.600005,0.471039,-0.732808,1.0,0.615097,...,0.016021,0.0,0.622103,0.479281,0.034743,0.0,0.625970,0.475309,0.035934,0.0
3,no handling,0.580567,0.507871,-0.741323,1.0,0.601659,0.473851,-0.655580,1.0,0.616745,...,0.014927,0.0,0.620831,0.484859,0.031953,0.0,0.624880,0.480526,0.032944,0.0
4,no handling,0.580704,0.510263,-0.829618,1.0,0.601796,0.476005,-0.740312,1.0,0.616884,...,0.015714,0.0,0.620579,0.487793,0.034107,0.0,0.624449,0.483998,0.035229,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,no handling,0.577269,0.565328,-0.680161,1.0,0.598697,0.522609,-0.599991,1.0,0.615084,...,0.010910,0.0,0.624726,0.527804,0.030475,0.0,0.628861,0.522378,0.031706,0.0
223,no handling,0.577788,0.565799,-0.732080,1.0,0.598793,0.523236,-0.646783,1.0,0.615187,...,0.010797,0.0,0.624379,0.528293,0.031181,0.0,0.628451,0.523318,0.032428,0.0
224,no handling,0.582799,0.561642,-0.873002,1.0,0.602520,0.520399,-0.776600,1.0,0.618848,...,0.011099,0.0,0.624894,0.529448,0.031113,0.0,0.628929,0.524340,0.032452,0.0
225,no handling,0.583142,0.554840,-0.735256,1.0,0.602776,0.514548,-0.648446,1.0,0.619229,...,0.012208,0.0,0.624077,0.525228,0.033405,0.0,0.627997,0.520846,0.034731,0.0


In [36]:
X = df.drop('class', axis=1) # 예측에 사용되는 특징벡터(여기서는 landmark의 좌표)를 x 에 저장
y = df['class'] # 우리가 예측하고자 하는 값을 y 값에 저장
#여기서 사용할 것은 multi classification임

In [37]:
# 수집한 dataset을 train dataset과 test set를 7:3으로 random하게 나눈다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [54]:
y_test

240    on handling
542    on handling
214    no handling
492    on handling
190    no handling
          ...     
387    on handling
370    on handling
173    no handling
590    on handling
451    on handling
Name: class, Length: 202, dtype: object

## 3.2 Train Machine Learning Classification Model

In [40]:
#make_pipeline(머신러닝 모델에 사용할 파이프라인 구축하는 모듈),StandardScaler(datsset을 표준화 하는 모듈) import
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [44]:
#위에서 정의한 pipeline에 사용할 모델들 구축(좋은 결과를 내기 위해 여러가지 모델 사용)
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [45]:
list(pipelines.values())[0] #처음 data가 표준화 된 후 logisticregression 모델을 사용해서 학습한다.

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [46]:
fit_models = {} 
#파이프 라인들 안의 모델들을 이용해서 학습한다.
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [47]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [43]:
fit_models['rc'].predict(X_test)

KeyError: 'rc'

## 3.3 Evaluate and Serialize Model 

In [48]:
from sklearn.metrics import accuracy_score # Accuracy metrics
#from sklearn.metrics accuracy, precision_recall
import pickle 

In [49]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) # model 별로 predict 한여 예측값을 yhat 변수에 저장한다. 
    print(algo, accuracy_score(y_test, yhat)) # accuracy 함수에 y_test(실제값),yhat(예측값)을 넣어 accuracy 계산

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [63]:
fit_models['rf'].predict(X_test)

array(['on handling', 'on handling', 'no handling', 'on handling',
       'no handling', 'no handling', 'no handling', 'on handling',
       'on handling', 'on handling', 'no handling', 'on handling',
       'no handling', 'on handling', 'on handling', 'on handling',
       'no handling', 'on handling', 'on handling', 'on handling',
       'no handling', 'on handling', 'on handling', 'on handling',
       'on handling', 'on handling', 'on handling', 'on handling',
       'no handling', 'on handling', 'on handling', 'no handling',
       'on handling', 'on handling', 'no handling', 'no handling',
       'on handling', 'on handling', 'no handling', 'on handling',
       'on handling', 'on handling', 'no handling', 'on handling',
       'no handling', 'on handling', 'no handling', 'on handling',
       'on handling', 'on handling', 'on handling', 'on handling',
       'on handling', 'on handling', 'on handling', 'on handling',
       'no handling', 'on handling', 'on handling', 'on handli

In [35]:
y_test

29     no handling
22     no handling
402    on handling
71     no handling
64     no handling
          ...     
420    on handling
153    no handling
346    on handling
101    no handling
366    on handling
Name: class, Length: 131, dtype: object

In [50]:
# body_language 파일명을 가진 pickle 파일을 이진법 쓰기 형태로 열어 randomforest로 했을 때의 결과를 저장한다.
with open('tesla.pkl', 'wb') as f: 
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [51]:
with open('tesla.pkl', 'rb') as f:
    model = pickle.load(f) # randomforest의 결과가 저장된 pickle 파일을 load 하여 내용을 model 에 저장한다.

In [52]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [59]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 #)
        
         #2. Right hand
        #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 #mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 #mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 #)

        # 3. Left Hand
        #mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 #mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 #mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 #)

        #4. Pose Detections
        
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 #mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 #mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 #)
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0] #landmark의 좌표값을 넣어서 운전상태를 나타내는 class를 body_language_class 변수에 할당
            body_language_prob = model.predict_proba(X)[0] # 예측한 운전상태(class) 정확도를 측정한다.
            print(body_language_class, body_language_prob)
            
            # 영상에 운전상태(class)와 class의 확률을 왼쪽 귀를 기준으로 동일한 곳에 표시하기 위해 왼쪽귀를 기준으로 잡는다.
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int)) #웹캠 frame에 크기를 맞춰준다.
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            # probobility는 최댓값을 출력하도록 한다. 
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

on handling [0.48 0.52]
no handling [0.58 0.42]
on handling [0.35 0.65]
on handling [0.25 0.75]
on handling [0.25 0.75]
on handling [0.25 0.75]
on handling [0.26 0.74]
on handling [0.29 0.71]
on handling [0.26 0.74]
on handling [0.25 0.75]
on handling [0.25 0.75]
on handling [0.25 0.75]
on handling [0.35 0.65]
on handling [0.37 0.63]
on handling [0.35 0.65]
on handling [0.36 0.64]
on handling [0.36 0.64]
on handling [0.35 0.65]
on handling [0.36 0.64]
on handling [0.36 0.64]
on handling [0.35 0.65]
on handling [0.34 0.66]
on handling [0.26 0.74]
on handling [0.24 0.76]
on handling [0.1 0.9]
on handling [0.12 0.88]
on handling [0.12 0.88]
on handling [0.17 0.83]
no handling [0.55 0.45]
no handling [0.56 0.44]
on handling [0.02 0.98]
on handling [0. 1.]
on handling [0. 1.]
on handling [0.27 0.73]
on handling [0.41 0.59]
on handling [0.45 0.55]
no handling [0.54 0.46]
no handling [0.54 0.46]
no handling [0.54 0.46]
no handling [0.54 0.46]
no handling [0.59 0.41]
no handling [0.71 0.29]
no